In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
DATA_DIR = Path('../data/')

df = pd.read_parquet('../generated_data/train_df.parquet')

In [9]:
# Get an example notebook
nb_id = df.index.unique('id')[6]
print('Notebook:', nb_id)

print("The disordered notebook:")
nb = df.loc[nb_id, :]
display(nb)
print()

Notebook: 00038c2941faa0
The disordered notebook:


,cell_type,source
cell_id,,
3e551fb7,code,# This Python 3 environment comes with many he...
45049ad8,code,"train_data = pd.read_csv(""/kaggle/input/titani..."
123b4f4c,code,import plotly.express as px
0b92cb59,code,train_data.head(20)
df963df4,code,train_data.isnull().sum() #checking out which...
0f3db81b,code,"px.bar(data_frame=train_data, x='Sex', y='Surv..."
33ff3073,code,total_passengers = train_data['Sex'].count()\n...
818c4c15,code,from sklearn.ensemble import RandomForestClass...
6cfbe868,markdown,## Survival Rate for Male Passenger is : 12.23...


In [10]:
df_orders = pd.read_csv(
    DATA_DIR / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

df_orders.head()

id
00001756c60be8    [1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b7...
00015c83e2717b    [2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c4172...
0001bdd4021779    [3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310...
0001daf4c2c76d    [97266564, a898e555, 86605076, 76cc2642, ef279...
0002115f48f982    [9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe...
Name: cell_order, dtype: object

In [11]:
# Get the correct order
cell_order = df_orders.loc[nb_id]

print("The ordered notebook:")
nb.loc[cell_order, :]

The ordered notebook:


,cell_type,source
cell_id,,
3e551fb7,code,# This Python 3 environment comes with many he...
45049ad8,code,"train_data = pd.read_csv(""/kaggle/input/titani..."
8bb41691,markdown,### Checking out the Titanic Dataset \n
123b4f4c,code,import plotly.express as px
0b92cb59,code,train_data.head(20)
5a8b6e2d,markdown,## EDA is all about asking the right questions...
df963df4,code,train_data.isnull().sum() #checking out which...
3c7d19bc,markdown,From the above inference Cabin needs to be eit...
0f3db81b,code,"px.bar(data_frame=train_data, x='Sex', y='Surv..."


In [12]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

cell_ranks = get_ranks(cell_order, list(nb.index))
nb.insert(0, 'rank', cell_ranks)

nb

,rank,cell_type,source
cell_id,,,
3e551fb7,0,code,# This Python 3 environment comes with many he...
45049ad8,1,code,"train_data = pd.read_csv(""/kaggle/input/titani..."
123b4f4c,3,code,import plotly.express as px
0b92cb59,4,code,train_data.head(20)
df963df4,6,code,train_data.isnull().sum() #checking out which...
0f3db81b,8,code,"px.bar(data_frame=train_data, x='Sex', y='Surv..."
33ff3073,10,code,total_passengers = train_data['Sex'].count()\n...
818c4c15,13,code,from sklearn.ensemble import RandomForestClass...
6cfbe868,11,markdown,## Survival Rate for Male Passenger is : 12.23...


In [13]:
df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)

ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

df_ranks = (
    pd.DataFrame
    .from_dict(ranks, orient='index')
    .rename_axis('id')
    .apply(pd.Series.explode)
    .set_index('cell_id', append=True)
)

df_ranks

rank
id             cell_id      
00001756c60be8 1862f0a6    0
               2a9e43d6    2
               038b763d    4
               2eefe0ef    6
               0beab1cd    8
...                      ...
fffe1d764579d5 0d770d6b   43
               d45ddc62   33
               1a63248d    0
               a8ffc8b4   69
               4e2d4c2d    3

[6370646 rows x 1 columns]

In [15]:
df_ancestors = pd.read_csv(DATA_DIR / 'train_ancestors.csv', index_col='id')
df_ancestors

,ancestor_id,parent_id
id,,
00001756c60be8,945aea18,NaN
00015c83e2717b,aa2da37e,317b65d12af9df
0001bdd4021779,a7711fde,NaN
0001daf4c2c76d,090152ca,NaN
0002115f48f982,272b483a,NaN
...,...,...
fffc30d5a0bc46,6aed207b,NaN
fffc3b44869198,a6aaa8d7,NaN
fffc63ff750064,0a1b5b65,NaN


In [19]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.1  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

# Split, keeping notebooks with a common origin (ancestor_id) together
ids = df.index.unique('id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

In [20]:
df_train.head()

cell_type  \
id             cell_id              
00001756c60be8 1862f0a6      code   
               2a9e43d6      code   
               038b763d      code   
               2eefe0ef      code   
               0beab1cd      code   

                                                                    source  
id             cell_id                                                      
00001756c60be8 1862f0a6  # This Python 3 environment comes with many he...  
               2a9e43d6  import numpy as np\nimport pandas as pd\nimpor...  
               038b763d  import warnings\nwarnings.filterwarnings('igno...  
               2eefe0ef      matplotlib.rcParams.update({'font.size': 14})  
               0beab1cd  def evaluate_preds(train_true_values, train_pr...

In [16]:
test_submission = pd.read_csv("../data/sample_submission.csv")
test_submission

,id,cell_order
0,0009d135ece78d,ddfd239c c6cd22db 1372ae9b 90ed07ab 7f388a41 2...
1,0010483c12ba9b,54c7cab3 fe66203e 7844d5f8 5ce8863c 4a0777c4 4...
2,0010a919d60e4f,aafc3d23 80e077ec b190ebb4 ed415c3c 322850af c...
3,0028856e09c5b7,012c9d02 d22526d1 3ae7ece3 eb293dfc
